##### Goal: Find out which game company has a higher overall rating

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
valve_html = requests.get('https://en.wikipedia.org/wiki/List_of_Valve_games')
blizzard_html = requests.get('https://en.wikipedia.org/wiki/List_of_Blizzard_Entertainment_games')

In [3]:
soup = bs(valve_html.content)

contents = soup.prettify()

#print(contents)

In [4]:
games = soup.select('.wikitable i a')
game_path = games[0]['href']

game_path

'/wiki/Half-Life_(video_game)'

In [16]:
def get_game_links(url):
    html = requests.get(url)
    soup = bs(html.content)
    games = soup.select('.wikitable i a')
    games = soup.select('.wikitable .fn i a')
    game_links = []
    base_path = 'https://en.wikipedia.org'

    for game in games:
        # subdirectory example: "/wiki/Half-Life_(video_game)"
        game_subdirectory = game['href']
        path = base_path + game_subdirectory
        game_links.append(path)
    
    return game_links

get_game_links('https://en.wikipedia.org/wiki/List_of_Valve_games')

['https://en.wikipedia.org/wiki/Half-Life_(video_game)',
 'https://en.wikipedia.org/wiki/Team_Fortress_Classic',
 'https://en.wikipedia.org/wiki/Counter-Strike_(video_game)',
 'https://en.wikipedia.org/wiki/Day_of_Defeat',
 'https://en.wikipedia.org/wiki/Counter-Strike:_Condition_Zero',
 'https://en.wikipedia.org/wiki/Counter-Strike:_Source',
 'https://en.wikipedia.org/wiki/Half-Life_2',
 'https://en.wikipedia.org/wiki/Half-Life_2:_Deathmatch',
 'https://en.wikipedia.org/wiki/Day_of_Defeat:_Source',
 'https://en.wikipedia.org/wiki/Half-Life_2:_Lost_Coast',
 'https://en.wikipedia.org/wiki/Half-Life_2:_Episode_One',
 'https://en.wikipedia.org/wiki/Half-Life_2:_Episode_Two',
 'https://en.wikipedia.org/wiki/Portal_(video_game)',
 'https://en.wikipedia.org/wiki/Team_Fortress_2',
 'https://en.wikipedia.org/wiki/The_Orange_Box',
 'https://en.wikipedia.org/wiki/Left_4_Dead',
 'https://en.wikipedia.org/wiki/Left_4_Dead_2',
 'https://en.wikipedia.org/wiki/Alien_Swarm',
 'https://en.wikipedia.org

In [6]:
import re

def remove_ref(contents):
    # regular expression to find all matches with format [c] where c is any character eg. [a] or [1]
    reference = r'\[.\]'
    
    if type(contents) == str:
        matches = re.findall(reference, contents)
        # for each match, remove it from the string
        for match in matches:
            contents = contents.replace(match, '')
        return contents
    
    elif type(contents) == list:
        contents_list = []
        for my_string in contents:
            matches = re.findall(reference, my_string)
            for match in matches:
                my_string = my_string.replace(match, '')
            contents_list.append(my_string)
        return contents_list

In [7]:
remove_ref(['sadsada[a]', 'dassdsadpas[a]'])

['sadsada', 'dassdsadpas']

In [8]:
def get_row_contents(row):
    """# to deal with platform
    if row.select('.plainlist ul li b'):
        full_list = row.select('.plainlist ul li')
        platform_date_list = []
        for i, item in enumerate(full_list):
            #print(item)
            #platform_name = "default platform"
            #date = "default date"
            if item.select('li b'):
                concat_platform = ""
                platforms_comma_separated = item.find_all('b')
                for j, platform in enumerate(platforms_comma_separated):
                    concat_platform += platform.get_text()
                    if (j < len(platforms_comma_separated)-1):
                        concat_platform += ", "
                platform_name = concat_platform
                #print(i, platform_name)
            else:
                date = item.get_text()
                #print(i, date)
            
            #print(i, platform_name, date)
            
            if (i % 2 == 1):
                platform_date_list.append(f'{platform_name}: {date}')
            #platform_date_list.append('something happening hahahahha')
        return platform_date_list"""
    # to handle releases
    # strip=True to remove whitespace chars (\n, xa0, etc)
    if row.select('li ul li'):
        return [li.get_text('', strip=True) for li in row.select('li ul li')]
    elif row.find('li'):
        return [li.get_text('', strip=True) for li in row.find_all('li')]
    #elif row.find('br'):
        #return [text for text in row.stripped_strings]
    elif "," in row.get_text():
        return [text.strip() for text in row.find('td', class_='infobox-data').get_text().split(',')]
    else:  
        return row.find('td', class_='infobox-data').get_text()

In [9]:
my_string = 'OS X, Linux'
my_string.split(',')

['OS X', ' Linux']

In [10]:
my_string = 'Single-player, multiplayer'
my_string.split(',')

['Single-player', ' multiplayer']

In [11]:
import re

def get_release_date(contents):
    date_re = r'[A-Za-z]+[ ]\d+,[ ]\d+'
    
    if type(contents) == str:
        match = re.search(date_re, contents)
        return match.group()
    elif type(contents) == list:
        for item in contents:
            match = re.search(date_re, item)
            if (match != None):
                return match.group()

get_release_date(['WW:September 26, 2005', 'WW:July 12, 2010', 'WW:March 14, 2013'])

'September 26, 2005'

In [12]:
def get_info_box(url):
    game_html = requests.get(url)
    game_soup = bs(game_html.content)
    
    info_box = game_soup.find('table', class_='infobox')
    info_rows = info_box.find_all('tr')
    
    game_info = {}
    
    for i, row in enumerate(info_rows):
        # get title from infobox
        if row.select('.infobox-above'):
            game_info['Name'] = row.find('th').get_text()
        # get image link from infobox
        elif row.select('.infobox-image'):
            image_link = base_path + row.find('a')['href']
            game_info['Image_link'] = image_link
        else:
            label = row.find('th', class_='infobox-label').get_text()
            contents = get_row_contents(row)
            if (label == 'Release'):
                contents = get_release_date(contents)
                
            contents = remove_ref(contents)
            game_info[label] = contents
    
    game_info['Source'] = url
    
    return game_info
    
    
get_info_box('https://en.wikipedia.org/wiki/Team_Fortress_Classic')

{'Name': 'Team Fortress Classic',
 'Image_link': 'https://en.wikipedia.org/wiki/File:Team_Fortress_Classic_box.jpg',
 'Developer(s)': 'Valve',
 'Publisher(s)': ['Sierra Studios', 'Valve (digital)'],
 'Designer(s)': ['John Cook', 'Robin Walker'],
 'Engine': 'GoldSrc',
 'Platform(s)': ['Microsoft Windows', 'OS X', 'Linux'],
 'Release': 'April 7, 1999',
 'Genre(s)': 'First-person shooter',
 'Mode(s)': 'Multiplayer',
 'Source': 'https://en.wikipedia.org/wiki/Team_Fortress_Classic'}

In [48]:
def scrape_game_links(game_links):
    games_info = []
    for i, link in enumerate(game_links):
        try:
            games_info.append(get_info_box(link))
            if (i % 10 == 0):
                print(f'--Scraped {i} games--')

        except Exception as e:
            print(link)
            print(e)

    print(f'Scraping finished with {i} games')

--Scraped 0 games--
--Scraped 10 games--
--Scraped 20 games--
Scraping finished with 26 games


In [49]:
games_info
# tf classic (release), 

[{'Name': 'Half-Life',
  'Image_link': 'https://en.wikipedia.org/wiki/File:Half-Life_Cover_Art.jpg',
  'Developer(s)': 'Valve',
  'Publisher(s)': 'Sierra Studios',
  'Writer(s)': 'Marc Laidlaw',
  'Composer(s)': 'Kelly Bailey',
  'Series': 'Half-Life',
  'Engine': 'GoldSrc',
  'Platform(s)': ['Windows', 'PlayStation 2', 'OS X', 'Linux'],
  'Release': 'November 19, 1998',
  'Genre(s)': 'First-person shooter',
  'Mode(s)': ['Single-player', 'multiplayer'],
  'Source': 'https://en.wikipedia.org/wiki/Half-Life_(video_game)'},
 {'Name': 'Team Fortress Classic',
  'Image_link': 'https://en.wikipedia.org/wiki/File:Team_Fortress_Classic_box.jpg',
  'Developer(s)': 'Valve',
  'Publisher(s)': ['Sierra Studios', 'Valve (digital)'],
  'Designer(s)': ['John Cook', 'Robin Walker'],
  'Engine': 'GoldSrc',
  'Platform(s)': ['Microsoft Windows', 'OS X', 'Linux'],
  'Release': 'April 7, 1999',
  'Genre(s)': 'First-person shooter',
  'Mode(s)': 'Multiplayer',
  'Source': 'https://en.wikipedia.org/wiki/Te

Things to clean:
- Remove references [a] [1] etc
- Convert from all li in one string to list of strings
- In release, format from '\nNovember 19, 1998\n Windows' to 'November 19, 1998 (Windows)'

In [13]:
my_string = 'Valve[1][2]'
remove_ref(my_string)

'Valve'

In [ ]:
my_string = '\nMatthew Armstrong\nBrian Hess\nRandy Pitchford\n'


In [13]:
games_info

[{'Name': 'Half-Life',
  'Image_link': 'https://en.wikipedia.org/wiki/File:Half-Life_Cover_Art.jpg',
  'Developer(s)': 'Valve[a]',
  'Publisher(s)': 'Sierra Studios',
  'Writer(s)': 'Marc Laidlaw',
  'Composer(s)': 'Kelly Bailey',
  'Series': 'Half-Life',
  'Engine': 'GoldSrc',
  'Platform(s)': 'WindowsPlayStation 2OS XLinux',
  'Release': '\nNovember 19, 1998\n Windows\n NA: November 19, 1998[2]EU: November 27, 1998[1]\n PlayStation 2\n NA: November 11, 2001EU: November 30, 2001\n OS X, Linux\n WW: February 14, 2013\n\n',
  'Genre(s)': 'First-person shooter',
  'Mode(s)': 'Single-player, multiplayer'},
 {'Name': 'Half-Life: Decay',
  'Image_link': 'https://en.wikipedia.org/wiki/File:Half-Life_Decay_title.jpg',
  'Developer(s)': 'Gearbox Software',
  'Publisher(s)': 'Sierra On-Line',
  'Designer(s)': 'Randy Pitchford[2]',
  'Writer(s)': '\nMatthew Armstrong\nBrian Hess\nRandy Pitchford\n',
  'Series': 'Half-Life',
  'Engine': 'GoldSrc',
  'Platform(s)': 'PlayStation 2',
  'Release': 'N

Tasks:
- Scrape both websites
- Convert to dictionary
- Convert to pandas
- Do some EDA
- Make insights